In [1]:
from aicspylibczi import CziFile
import numpy as np
import skimage as sk
import pandas as pd
import os
from glob import glob
import napari
from cellpose import models, io, utils
#import pyclesperanto as cle


In [26]:
def read_and_prep_images(czifile):
    scenes = [czifile.read_image(S=s)[0] for s in range(czifile.size[2])]
    squeezed = [np.squeeze(i) for i in scenes]
    reorder = [np.moveaxis(i,0,-1) for i in squeezed] #channel axis needs to be placed last for scikit-image
    nuclei_channel = [reorder[i][...,-1] for i in range(len(reorder))]
    gauss_nuc = [sk.filters.gaussian(img, sigma=3) for img in nuclei_channel]
    return scenes, reorder, gauss_nuc, nuclei_channel        


In [ ]:
def segment_and_measure_CP(reorder, gauss_nuc, props, save_path, img_name):
    for i in range(len(gauss_nuc)):
        model = models.Cellpose(gpu=True,model_type='nuclei') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
        channels =[[0,0]]
        masks, _, _, _ = model.eval(gauss_nuc[i], diameter=150, channels=channels, z_axis=0,stitch_threshold=0.1,flow_threshold=0.4,cellprob_threshold=0.0, min_size=100)
        # filtered_masks = sk.segmentation.clear_border(masks) #nuclei span the whole image in Z and are removed even if they are complete objects
        df = sk.measure.regionprops_table(masks,reorder[i],properties=props)
        df = pd.DataFrame.from_dict(df)
        #Save Everything
        dataframe_path = os.path.join(save_path,'measurements')
        df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'_scene0'+str(i+1)+'.csv'))
        masks_path = os.path.join(save_path,'masks')
        sk.io.imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'_scene0'+str(i+1)+'.tif'),masks,check_contrast=False)
        scenes_path = os.path.join(save_path,'scenes')
        sk.io.imsave(os.path.join(scenes_path, img_name[:-4]+'_scene0'+str(i+1)+'.tif'),reorder[i])

In [28]:
def save(save_path, img_name, squeezed, gauss_nuc, filtered_masks, df):
    try:
        scenes_path = os.mkdir(os.path.join(save_path,'scenes'))
        gauss_path = os.mkdir(os.path.join(save_path,'gauss_nuc'))
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        gauss_path = os.path.join(save_path,'gauss_nuc')
        scenes_path = os.path.join(save_path,'scenes')
        dataframe_path = os.path.join(save_path,'measurements')
    for i in range(len(filtered_masks)):
        sk.io.imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'_scene'+str(i)+'.tif'),filtered_masks[i],check_contrast=False)
        sk.io.imsave(os.path.join(scenes_path,'normalized_'+img_name[:-4]+'_scene'+str(i)+'.tif'),squeezed[i])
        sk.io.imsave(os.path.join(gauss_path,'normalized_'+img_name[:-4]+'_scene'+str(i)+'.tif'),gauss_nuc[i])
        df[i].to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'_scene'+str(i)+'.csv'))

In [29]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))
save_path = "E:/Fondufe-Mittendorf_Lab/OIC-150_SATB2-GFP_circ3915-3XFLAG_Fluorescence_Quantification/"
img_name = os.path.basename(file_paths[0])

scenes, reorder, gauss_nuc, nuclei_channel = read_and_prep_images(czifile=czi_files[0])
props = ('label', 'intensity_mean')
segment_and_measure_CP(reorder, gauss_nuc, props,save_path,img_name)

100%|██████████| 23/23 [00:00<00:00, 120.26it/s]
c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: E:/Fondufe-Mittendorf_Lab/OIC-150_SATB2-GFP_circ3915-3XFLAG_Fluorescence_Quantification/scenes\Slide1-unstained_2025_05_05__15_40_55_Airyscan Processing_scene03.tif is a low contrast image
  return func(*args, **kwargs)


In [22]:
reorder[0].shape
#sk.io.imsave(os.path.join(save_path, 'scenes','test.tif'),squeezed[0])

(24, 1012, 1012, 4)

In [6]:
save_path = "E:/Fondufe-Mittendorf_Lab/OIC-150_SATB2-GFP_circ3915-3XFLAG_Fluorescence_Quantification/"
img_name = os.path.basename(file_paths[0])
save(save_path,img_name,squeezed,gauss_nuc,filtered_masks,df)

c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: E:/Fondufe-Mittendorf_Lab/OIC-150_SATB2-GFP_circ3915-3XFLAG_Fluorescence_Quantification/scenes\normalized_Slide1-unstained_2025_05_05__15_40_55_Airyscan Processing_scene2.tif is a low contrast image
  return func(*args, **kwargs)


In [14]:
gauss_nuc[0].shape

(24, 1012, 1012)

In [ ]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))


In [ ]:
czi_files[0].size[2]

In [ ]:
img = [czi_files[0].read_image(S=i)[0] for i in range(czi_files[0].size[2])]
squeezed = [np.squeeze(i) for i in img]

In [ ]:
squeezed[0].shape

In [ ]:
img = np.squeeze(img)
print(img.shape)
split = np.split(img, img.shape[1], axis=1)

In [ ]:
squeeze = np.squeeze(split)
squeeze[0].shape

In [ ]:
gauss = sk.filters.gaussian(test[-1,...],sigma=3)

In [ ]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='nuclei') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(gauss, diameter=150, channels=channels, z_axis=0,stitch_threshold=0.1,flow_threshold=0.4,cellprob_threshold=0.0, min_size=100)

In [ ]:
voxel = [0.25,0.13,0.13]
viewer = napari.view_image(gauss, scale=voxel)
viewer.add_labels(masks,scale=voxel)